# BigQueryML example


### Scenario
Data flowing from PI Integrator to BigQuery table, create a model and operationalize
hydraulic dataset care of https://archive.ics.uci.edu/ml/datasets/Condition+monitoring+of+hydraulic+systems

***

## setup - tables/views
hydraulic - table - PI Integrator view writes data every 5 minutes<br>
hydraulicv - view - average each instance dataset to one row<br>
hydraulic_instances_cooler_condition - table - specify a window the defines the dataset shape (2205 instances) and filter out invalid labels<br>
hydraulic_instances_cooler_conditionv - view - based upon table with similar name, remove columns not required for modeling<br>

***

## configuration

In [1]:
# install/upgrade bigquery client
!pip install --upgrade 'google-cloud-bigquery[bqstorage,pandas]'

     |████████████████████████████████| 201 kB 7.6 MB/s eta 0:00:01
  Attempting uninstall: google-cloud-bigquery
    Found existing installation: google-cloud-bigquery 2.25.1
    Uninstalling google-cloud-bigquery-2.25.1:
      Successfully uninstalled google-cloud-bigquery-2.25.1


In [2]:
# load/verify magic commands
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [3]:
%%bigquery
# query table being populated from PI with PI Integrator (verify connection)
select *
from `aveva-gcp-accelerator-dev.hydraulic.hydraulic`
limit 1

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.46s/rows]


,Hydraulic_Test_Rig,TimeStamp,Cooler_Condition,Cooling_Efficiency,Cooling_Power,Efficiency_Factor,Hydraulic_Accumulator,Internal_Pump_Leakage,Motor_Power,Pressure1,...,Temperature1,Temperature2,Temperature3,Temperature4,Valve_Condition,Vibration,Volume_Flow1,Volume_Flow2,PIIntTSTicks,PIIntShapeID
0,Hydraulic Test Rig,2021-09-01 09:17:36+00:00,100.0,19.478,1.528,10.627,90.0,0.0,2846.6,188.43,...,56.586,59.551,58.02,51.73,100.0,0.777,1.1,9.0,637660846560000000,1


***

### create supporting tables and views to create model

In [4]:
%%bigquery
# create a view to represent each instance as one row to simplify creating an example model
create view if not exists  `aveva-gcp-accelerator-dev.hydraulic.hydraulicv` as
select count(*) as count,
    CAST(FORMAT_TIMESTAMP("%Y%m%d%H%M",TimeStamp) as NUMERIC) as cycle_id, min(timestamp_trunc(TimeStamp,MINUTE)) as cycle,
    avg(Cooler_Condition) as Cooler_Condition,
    avg(Cooling_Efficiency) as Cooling_Efficiency,
    avg(Cooling_Power) as Cooling_Power,
    avg(Efficiency_Factor) as Efficiency_Factor,
    avg(Hydraulic_Accumulator) as Hydraulic_Accumulator,
    avg(Internal_Pump_Leakage) as Internal_Pump_Leakage,
    avg(Motor_Power) as Motor_Power,
    avg(Pressure1) as Pressure1,
    avg(Pressure2) as Pressure2,
    avg(Pressure3) as Pressure3,
    avg(Pressure4) as Pressure4,
    avg(Pressure5) as Pressure5,
    avg(Pressure6) as Pressure6,
    avg(Stable) as Stable,
    avg(Temperature1) as Temperature1,
    avg(Temperature2) as Temperature2,
    avg(Temperature3) as Temperature3,
    avg(Temperature4) as Temperature4,
    avg(Valve_Condition) as Valve_Condition,
    avg(Vibration) as Vibration,
    avg(Volume_Flow1) as Volume_Flow1,
    avg(Volume_Flow2) as Volume_Flow2
from hydraulic.hydraulic
group by cycle_id
order by cycle_id

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 1033.84query/s]


""


In [5]:
%%bigquery
# create a table with the source dataset (2205 instances) as input to build a model
create table if not exists `aveva-gcp-accelerator-dev.hydraulic.hydraulic_instances_cooler_condition`
options (
    description='table contains all instances from original dataset with valid cooler condition values'
) as 
select *
from  `aveva-gcp-accelerator-dev.hydraulic.hydraulicv`
where (cycle >= timestamp "2021-08-28"
       and 
       cycle < timestamp_add(timestamp "2021-08-28", interval 2205 minute))
and Cooler_Condition in (1,3,20,100) /* clean-up dataset to remove invalid labels*/

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 1091.98query/s]


""


In [6]:
%%bigquery df
# verify dataset
select *
from `aveva-gcp-accelerator-dev.hydraulic.hydraulic_instances_cooler_condition`

Downloading: 100%|██████████| 2201/2201 [00:01<00:00, 1319.02rows/s]


In [7]:
df.head()

,count,cycle_id,cycle,Cooler_Condition,Cooling_Efficiency,Cooling_Power,Efficiency_Factor,Hydraulic_Accumulator,Internal_Pump_Leakage,Motor_Power,...,Pressure6,Stable,Temperature1,Temperature2,Temperature3,Temperature4,Valve_Condition,Vibration,Volume_Flow1,Volume_Flow2
0,60,280058.000000000,2021-08-28 00:58:00+00:00,100.0,27.273033,2.119633,59.835233,90.0,0.0,2384.133500,...,8.457833,0.0,54.833700,59.833500,56.918067,48.223683,100.0,0.611650,6.572000,9.045167
1,60,280318.000000000,2021-08-28 03:18:00+00:00,100.0,26.508517,1.721867,59.537483,90.0,0.0,2452.101833,...,8.933333,0.0,46.592300,51.574217,48.744000,42.096033,100.0,0.703767,6.641167,9.614500
2,60,280355.000000000,2021-08-28 03:55:00+00:00,100.0,26.638850,1.725717,59.851333,90.0,0.0,2441.127000,...,8.919667,0.0,46.445233,51.435767,48.664717,42.005183,100.0,0.601533,6.638500,9.614167
3,60,280519.000000000,2021-08-28 05:19:00+00:00,20.0,26.760883,1.701383,57.531900,130.0,1.0,2463.759500,...,8.965833,0.0,45.941567,50.780117,48.084133,41.538133,80.0,0.633033,6.456000,9.646667
4,60,280728.000000000,2021-08-28 07:28:00+00:00,20.0,27.304317,1.726750,56.793583,115.0,1.0,2467.523333,...,9.035833,0.0,44.827717,49.782933,47.028633,40.413483,73.0,0.616317,6.434500,9.686833


In [8]:
%%bigquery
# create a view to remove columns not required for modeling
create or replace view `aveva-gcp-accelerator-dev.hydraulic.hydraulic_instances_cooler_conditionv` as 
select * except (count,cycle_id,cycle)
from `aveva-gcp-accelerator-dev.hydraulic.hydraulic_instances_cooler_condition`

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 1217.86query/s]


""


In [9]:
%%bigquery dfv
# view the view!
select * from `aveva-gcp-accelerator-dev.hydraulic.hydraulic_instances_cooler_conditionv`

Downloading: 100%|██████████| 2201/2201 [00:01<00:00, 1385.91rows/s]


In [10]:
dfv.head()

,Cooler_Condition,Cooling_Efficiency,Cooling_Power,Efficiency_Factor,Hydraulic_Accumulator,Internal_Pump_Leakage,Motor_Power,Pressure1,Pressure2,Pressure3,...,Pressure6,Stable,Temperature1,Temperature2,Temperature3,Temperature4,Valve_Condition,Vibration,Volume_Flow1,Volume_Flow2
0,100.0,27.273033,2.119633,59.835233,90.0,0.0,2384.133500,156.514000,105.791833,1.741167,...,8.457833,0.0,54.833700,59.833500,56.918067,48.223683,100.0,0.611650,6.572000,9.045167
1,100.0,26.508517,1.721867,59.537483,90.0,0.0,2452.101833,158.308667,107.323333,1.776500,...,8.933333,0.0,46.592300,51.574217,48.744000,42.096033,100.0,0.703767,6.641167,9.614500
2,100.0,26.638850,1.725717,59.851333,90.0,0.0,2441.127000,158.345000,107.381167,1.787667,...,8.919667,0.0,46.445233,51.435767,48.664717,42.005183,100.0,0.601533,6.638500,9.614167
3,20.0,26.760883,1.701383,57.531900,130.0,1.0,2463.759500,158.278333,107.031500,1.740500,...,8.965833,0.0,45.941567,50.780117,48.084133,41.538133,80.0,0.633033,6.456000,9.646667
4,20.0,27.304317,1.726750,56.793583,115.0,1.0,2467.523333,158.602333,106.915833,1.748833,...,9.035833,0.0,44.827717,49.782933,47.028633,40.413483,73.0,0.616317,6.434500,9.686833


***

### create a model and evaluate

In [131]:
%%bigquery 
create or replace model `aveva-gcp-accelerator-dev.hydraulic.hydraulic_model_xg_001`
options (
    model_type='BOOSTED_TREE_CLASSIFIER'
    ,BOOSTER_TYPE = 'GBTREE'
    ,NUM_PARALLEL_TREE = 1
    /* ,MAX_INTERATIONS = 50 */
    /* ,TREE_METHOD = 'HIST' for larger datasets */
    ,EARLY_STOP = FALSE
    ,SUBSAMPLE = 0.8
    ,ENABLE_GLOBAL_EXPLAIN = TRUE /* for use with ML.global_explain */
    ,INPUT_LABEL_COLS = ['cooler_condition']) as
select * from `aveva-gcp-accelerator-dev.hydraulic.hydraulic_instances_cooler_conditionv`

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1661.77query/s]                        


""


In [11]:
%%bigquery
/* Confusion matrix
_____________________________________________________________
Actual result\test result | True           | False
-------------------------------------------------------------
True                      | True Positive  | False Negative
False                     | False Positive | True Negative
_____________________________________________________________
Recall – horizonal TP / (TP + FN) - completeness, not miss any failures
Accuracy – diagonal (TP + TN)/total - % correct
Precision – vertical TP / (TP + FP) - quality, minimize false alarms
*/
select *
from ML.EVALUATE(MODEL `aveva-gcp-accelerator-dev.hydraulic.hydraulic_model_xg_001`)

Downloading: 100%|██████████| 1/1 [00:02<00:00,  2.07s/rows]


,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,1.0,0.996764,0.997619,0.998379,0.010686,1.0


In [12]:
%%bigquery
select * 
from ML.TRAINING_INFO(MODEL `aveva-gcp-accelerator-dev.hydraulic.hydraulic_model_xg_001`)

Downloading: 100%|██████████| 20/20 [00:01<00:00, 12.81rows/s]


,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,20,0.002069,0.010686,0.3,57
1,0,19,0.002484,0.010984,0.3,101
2,0,18,0.003087,0.011047,0.3,71
3,0,17,0.003894,0.011338,0.3,48
4,0,16,0.004981,0.011824,0.3,69
5,0,15,0.006267,0.013089,0.3,45
6,0,14,0.008138,0.014370,0.3,51
7,0,13,0.010584,0.016318,0.3,12979
8,0,12,0.013759,0.019528,0.3,61
9,0,11,0.018250,0.023489,0.3,57


In [13]:
%%bigquery
select * 
from ML.GLOBAL_EXPLAIN(MODEL `aveva-gcp-accelerator-dev.hydraulic.hydraulic_model_xg_001`)

Downloading: 100%|██████████| 21/21 [00:01<00:00, 11.37rows/s]


,feature,attribution
0,Pressure2,1.083194
1,Temperature4,0.924750
2,Temperature1,0.391268
3,Temperature2,0.221720
4,Cooling_Efficiency,0.101549
5,Hydraulic_Accumulator,0.090976
6,Motor_Power,0.089898
7,Pressure1,0.062523
8,Pressure6,0.045265
9,Pressure3,0.039288


### execute model inference

In [14]:
%%bigquery
# predict equipment status
select *
from ml.predict(MODEL `aveva-gcp-accelerator-dev.hydraulic.hydraulic_model_xg_001`,
                (select * from `aveva-gcp-accelerator-dev.hydraulic.hydraulic_instances_cooler_conditionv` limit 1))

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.38s/rows]


,predicted_cooler_condition,predicted_cooler_condition_probs,Cooler_Condition,Cooling_Efficiency,Cooling_Power,Efficiency_Factor,Hydraulic_Accumulator,Internal_Pump_Leakage,Motor_Power,Pressure1,...,Pressure6,Stable,Temperature1,Temperature2,Temperature3,Temperature4,Valve_Condition,Vibration,Volume_Flow1,Volume_Flow2
0,100.0,"[{'label': 100.0, 'prob': 0.997282862663269}, ...",100.0,27.273033,2.119633,59.835233,90.0,0.0,2384.1335,156.514,...,8.457833,0.0,54.8337,59.8335,56.918067,48.223683,100.0,0.61165,6.572,9.045167


# simulate operationalizing a model inference

In [15]:
%%bigquery 
# retrieve latest record
select * 
from `aveva-gcp-accelerator-dev.hydraulic.hydraulicv`
order by cycle desc
limit 1

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.55s/rows]


,count,cycle_id,cycle,Cooler_Condition,Cooling_Efficiency,Cooling_Power,Efficiency_Factor,Hydraulic_Accumulator,Internal_Pump_Leakage,Motor_Power,...,Pressure6,Stable,Temperature1,Temperature2,Temperature3,Temperature4,Valve_Condition,Vibration,Volume_Flow1,Volume_Flow2
0,2,30009.000000000,2021-09-03 00:09:00+00:00,100.0,19.1645,1.5105,32.4925,90.0,0.0,2625.325,...,8.365,0.0,57.5215,60.922,58.961,52.6855,100.0,0.702,0.52,8.945


In [16]:
%%bigquery
# predict cooler condition
select *
from ml.predict(MODEL `aveva-gcp-accelerator-dev.hydraulic.hydraulic_model_xg_001`,
                (select * from `aveva-gcp-accelerator-dev.hydraulic.hydraulicv` order by cycle desc limit 1))

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.33s/rows]


,predicted_cooler_condition,predicted_cooler_condition_probs,count,cycle_id,cycle,Cooler_Condition,Cooling_Efficiency,Cooling_Power,Efficiency_Factor,Hydraulic_Accumulator,...,Pressure6,Stable,Temperature1,Temperature2,Temperature3,Temperature4,Valve_Condition,Vibration,Volume_Flow1,Volume_Flow2
0,100.0,"[{'label': 100.0, 'prob': 0.99835205078125}, {...",60,30014.000000000,2021-09-03 00:14:00+00:00,100.0,19.135767,1.506467,1.0829,90.0,...,8.352167,0.0,57.423917,60.7257,59.06975,52.7772,100.0,0.737433,3.2805,8.913833


## simulate inference and write results to prediction table

In [17]:
%%bigquery
# table to store inference results
CREATE TABLE IF NOT EXISTS
  `osi-pi-gcp-accelerator.hydraulic.hydraulic_predictions` ( timestamp timestamp,
    prediction STRUCT< stable FLOAT64,
    cooler float64,
    accumulator float64,
    pump_leakage float64,
    valve float64> );

Query complete after 0.00s: 100%|██████████| 1/1 [00:00<00:00, 1239.45query/s]


""


In [18]:
%%bigquery
# run inference for specified period to cover late arriving data
MERGE
  `aveva-gcp-accelerator-dev.hydraulic.hydraulic_predictions` P
USING
  (
  SELECT
    cycle AS timestamp,
    STRUCT(0.0,
      predicted_cooler_condition,
      0.0,
      0.0,
      0.0) AS prediction
  FROM
    ML.PREDICT(MODEL `aveva-gcp-accelerator-dev.hydraulic.hydraulic_model_xg_001`,
      (
      SELECT
        *
      FROM
        `aveva-gcp-accelerator-dev.hydraulic.hydraulicv`
      WHERE
        cycle_id < CAST(FORMAT_TIMESTAMP("%Y%m%d%H%M",TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 1 MINUTE)) AS NUMERIC)
        AND cycle_id > CAST(FORMAT_TIMESTAMP("%Y%m%d%H%M",TIMESTAMP_SUB(CURRENT_TIMESTAMP(), INTERVAL 6 MINUTE)) AS NUMERIC))) AS model) T
ON
  P.timestamp = T.timestamp
  WHEN NOT MATCHED THEN INSERT (timestamp, prediction) VALUES (timestamp,prediction) 

Query complete after 0.00s: 100%|██████████| 6/6 [00:00<00:00, 3428.12query/s]                        


""


In [19]:
%%bigquery
# view inference results in output table
select *
from `aveva-gcp-accelerator-dev.hydraulic.hydraulic_predictions`
order by timestamp desc
limit 10

Downloading: 100%|██████████| 10/10 [00:01<00:00,  6.53rows/s]


,timestamp,prediction
0,2021-09-03 00:13:00+00:00,"{'stable': 0.0, 'cooler': 100.0, 'accumulator'..."
1,2021-09-03 00:12:00+00:00,"{'stable': 0.0, 'cooler': 100.0, 'accumulator'..."
2,2021-09-03 00:11:00+00:00,"{'stable': 0.0, 'cooler': 100.0, 'accumulator'..."
3,2021-09-03 00:10:00+00:00,"{'stable': 0.0, 'cooler': 100.0, 'accumulator'..."
4,2021-09-01 05:24:00+00:00,"{'stable': 0.0, 'cooler': 100.0, 'accumulator'..."
5,2021-09-01 05:23:00+00:00,"{'stable': 0.0, 'cooler': 100.0, 'accumulator'..."
6,2021-09-01 05:22:00+00:00,"{'stable': 0.0, 'cooler': 100.0, 'accumulator'..."
7,2021-09-01 05:21:00+00:00,"{'stable': 0.0, 'cooler': 100.0, 'accumulator'..."
8,2021-04-25 06:44:00+00:00,"{'stable': 0.0, 'cooler': 100.0, 'accumulator'..."
9,2021-04-25 06:43:00+00:00,"{'stable': 0.0, 'cooler': 100.0, 'accumulator'..."


In [147]:
%%bigquery
# compare actual to predicted
select cycle as timestamp, cooler_condition, prediction.cooler as coller_condition_prediction
from `aveva-gcp-accelerator-dev.hydraulic.hydraulicv` a, `aveva-gcp-accelerator-dev.hydraulic.hydraulic_predictions` b
where a.cycle = b.timestamp
order by cycle desc
limit 5

Downloading: 100%|██████████| 5/5 [00:01<00:00,  3.29rows/s]


,timestamp,cooler_condition,coller_condition_prediction
0,2021-09-03 00:13:00+00:00,100.0,100.0
1,2021-09-03 00:12:00+00:00,100.0,100.0
2,2021-09-03 00:11:00+00:00,100.0,100.0
3,2021-09-03 00:10:00+00:00,100.0,100.0
4,2021-09-01 05:24:00+00:00,100.0,100.0


## operationalize model

using the 'run inference for specified period to cover late arriving data' query above create a scheduled task using the BigQuery console to run every 15 minutes

***

## write values to PI from BigQuery

### create a file called hydraulic_result.sql and store in RDBMSPI\SQL directory:
select 
timestamp as PI_TIMESTAMP,
prediction.cooler as PI_VALUE1,
0 as PI_STATUS1
from `osi-pi-gcp-accelerator.hydraulic.hydraulic_predictions`
where timestamp > ? 
order by timestamp asc;

### Install and configure 
1. ODBC (64-bit) driver for Bigquery
2. PI Interface for RDBMS
3. Configure PI Point(s) to store prediction
- PI Point attributes
- Location1-3: 1
- Location4: 4
- InstrumentTag: hydraulic_result.sql
- Exdesc: P1=TS
- Point Type: Float32